In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import time

class NaverLandScraper:
    def __init__(self):
        # Selenium 설정
        self.options = webdriver.ChromeOptions()
        self.options.add_argument('--start-maximized')
        
    def scrape_data(self, search_term):
        if not search_term:
            print("검색어를 입력해주세요.")
            return
            
        try:
            # 웹드라이버 시작
            print("브라우저를 시작합니다...")
            driver = webdriver.Chrome(options=self.options)
            driver.get("https://land.naver.com/")
            
            # 매물 메뉴 클릭
            print("매물 페이지로 이동합니다...")
            WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.CSS_SELECTOR, "a[href='https://new.land.naver.com/complexes']"))
            ).click()
            
            # 검색창 입력
            print(f"'{search_term}' 검색 중...")
            search_box = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.ID, "land_search"))
            )
            search_box.send_keys(search_term)
            search_box.send_keys(Keys.RETURN)
            
            # 데이터 수집을 위한 대기
            print("검색 결과를 불러오는 중...")
            time.sleep(3)
            
            # 매물 정보 수집
            print("매물 정보를 수집하는 중...")
            items = driver.find_elements(By.CSS_SELECTOR, ".item_inner")
            data = []
            
            for item in items:
                try:
                    # 동 정보
                    title = item.find_element(By.CSS_SELECTOR, ".item_title .text").text
                    
                    # 가격 정보
                    price_element = item.find_element(By.CSS_SELECTOR, ".price_line")
                    price_type = price_element.find_element(By.CSS_SELECTOR, ".type").text
                    price = price_element.find_element(By.CSS_SELECTOR, ".price").text
                    
                    # 상세 정보
                    spec = item.find_element(By.CSS_SELECTOR, ".spec").text
                    specs = spec.split(", ")
                    
                    # 면적과 층수 추출
                    area = specs[0] if specs else "정보없음"
                    floor = specs[1] if len(specs) > 1 else "정보없음"
                    
                    data.append({
                        "동": title,
                        "거래유형": price_type,
                        "가격": price,
                        "면적": area,
                        "층": floor
                    })
                except:
                    continue
            
            # DataFrame 생성 및 엑셀 저장
            df = pd.DataFrame(data)
            file_name = f"naver_land_{search_term}_{time.strftime('%Y%m%d_%H%M%S')}.xlsx"
            df.to_excel(file_name, index=False)
            
            print(f"\n수집 완료! 데이터가 {file_name}에 저장되었습니다.")
            print(f"총 {len(data)}개의 매물 정보가 저장되었습니다.\n")
            
            driver.quit()
            
        except Exception as e:
            print(f"오류가 발생했습니다: {str(e)}")
            try:
                driver.quit()
            except:
                pass

def main():
    scraper = NaverLandScraper()
    while True:
        search_term = input("\n검색어를 입력하세요 (종료하려면 'q' 입력): ")
        if search_term.lower() == 'q':
            print("프로그램을 종료합니다.")
            break
        scraper.scrape_data(search_term)

if __name__ == "__main__":
    main()

In [4]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import time

class NaverLandScraper:
    def __init__(self):
        self.options = webdriver.ChromeOptions()
        self.options.add_argument('--start-maximized')
        
    def scroll_to_bottom(self, driver):
        SCROLL_PAUSE_TIME = 1.5
        
        # 현재 스크롤 높이 가져오기
        last_height = driver.execute_script("return document.documentElement.scrollHeight")
        
        while True:
            # 스크롤 맨 아래로 내리기
            driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight);")
            
            # 페이지 로딩 대기
            time.sleep(SCROLL_PAUSE_TIME)
            
            # 새로운 스크롤 높이 계산
            new_height = driver.execute_script("return document.documentElement.scrollHeight")
            
            # 새로운 매물이 로딩되지 않았다면 종료
            if new_height == last_height:
                break
                
            last_height = new_height
            print("스크롤 중... 더 많은 매물을 불러오는 중...")

    def scrape_data(self, search_term):
        if not search_term:
            print("검색어를 입력해주세요.")
            return
            
        try:
            print("브라우저를 시작합니다...")
            driver = webdriver.Chrome(options=self.options)
            driver.get("https://land.naver.com/")
            
            print("매물 페이지로 이동합니다...")
            WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.CSS_SELECTOR, "a[href='https://new.land.naver.com/complexes']"))
            ).click()
            
            print(f"'{search_term}' 검색 중...")
            search_box = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.ID, "land_search"))
            )
            search_box.send_keys(search_term)
            search_box.send_keys(Keys.RETURN)
            
            print("검색 결과를 불러오는 중...")
            time.sleep(3)
            
            # 모든 매물을 로딩하기 위해 스크롤
            self.scroll_to_bottom(driver)
            
            print("매물 정보를 수집하는 중...")
            items = driver.find_elements(By.CSS_SELECTOR, ".item_inner")
            data = []
            
            for idx, item in enumerate(items, 1):
                try:
                    # 동 정보
                    title = item.find_element(By.CSS_SELECTOR, ".item_title .text").text
                    
                    # 가격 정보
                    price_element = item.find_element(By.CSS_SELECTOR, ".price_line")
                    price_type = price_element.find_element(By.CSS_SELECTOR, ".type").text
                    price = price_element.find_element(By.CSS_SELECTOR, ".price").text
                    
                    # 상세 정보
                    spec = item.find_element(By.CSS_SELECTOR, ".spec").text
                    specs = spec.split(", ")
                    
                    # 면적과 층수 추출
                    area = specs[0] if specs else "정보없음"
                    floor = specs[1] if len(specs) > 1 else "정보없음"
                    
                    data.append({
                        "동": title,
                        "거래유형": price_type,
                        "가격": price,
                        "면적": area,
                        "층": floor
                    })
                    
                    print(f"\r현재 {idx}개의 매물 정보 수집 중...", end="")
                    
                except:
                    continue
            
            print("\n데이터 정리 중...")
            df = pd.DataFrame(data)
            file_name = f"naver_land_{search_term}_{time.strftime('%Y%m%d_%H%M%S')}.xlsx"
            df.to_excel(file_name, index=False)
            
            print(f"\n수집 완료! 데이터가 {file_name}에 저장되었습니다.")
            print(f"총 {len(data)}개의 매물 정보가 저장되었습니다.\n")
            
            driver.quit()
            
        except Exception as e:
            print(f"오류가 발생했습니다: {str(e)}")
            try:
                driver.quit()
            except:
                pass

def main():
    scraper = NaverLandScraper()
    while True:
        search_term = input("\n검색어를 입력하세요 (종료하려면 'q' 입력): ")
        if search_term.lower() == 'q':
            print("프로그램을 종료합니다.")
            break
        scraper.scrape_data(search_term)

if __name__ == "__main__":
    main()


검색어를 입력하세요 (종료하려면 'q' 입력):  도곡렉슬


브라우저를 시작합니다...
매물 페이지로 이동합니다...
'도곡렉슬' 검색 중...
검색 결과를 불러오는 중...
매물 정보를 수집하는 중...
현재 20개의 매물 정보 수집 중...
데이터 정리 중...

수집 완료! 데이터가 naver_land_도곡렉슬_20241210_070257.xlsx에 저장되었습니다.
총 20개의 매물 정보가 저장되었습니다.



KeyboardInterrupt: Interrupted by user

In [6]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import time

class NaverLandScraper:
    def __init__(self):
        self.options = webdriver.ChromeOptions()
        self.options.add_argument('--start-maximized')
        
    def scroll_down(self, driver):
        # 초기 스크롤 위치
        last_height = driver.execute_script("return document.body.scrollHeight")
        
        while True:
            # 맨 아래로 스크롤
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            
            # 새로운 내용 로딩 대기
            time.sleep(2)
            
            # 새로운 스크롤 높이
            new_height = driver.execute_script("return document.body.scrollHeight")
            
            # 더 이상 새로운 내용이 없으면 종료
            if new_height == last_height:
                break
                
            last_height = new_height
            print("스크롤 다운... 더 많은 매물을 불러오는 중...")
            
    def wait_for_items(self, driver):
        try:
            loading_items = driver.find_elements(By.CSS_SELECTOR, ".item_inner.is-loading")
            if loading_items:
                time.sleep(2)  # 로딩 대기
        except:
            pass

    def scrape_data(self, search_term):
        if not search_term:
            print("검색어를 입력해주세요.")
            return
            
        try:
            print("브라우저를 시작합니다...")
            driver = webdriver.Chrome(options=self.options)
            driver.get("https://land.naver.com/")
            
            # 매물 메뉴 클릭
            print("매물 페이지로 이동합니다...")
            WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.CSS_SELECTOR, "a[href='https://new.land.naver.com/complexes']"))
            ).click()
            
            # 검색창 입력
            print(f"'{search_term}' 검색 중...")
            search_box = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.ID, "land_search"))
            )
            search_box.send_keys(search_term)
            search_box.send_keys(Keys.RETURN)
            
            # 데이터 로딩 대기
            time.sleep(3)
            
            # 전체 매물 로딩을 위한 스크롤
            print("전체 매물을 불러오는 중...")
            self.scroll_down(driver)
            
            # 매물 정보 수집
            print("매물 정보를 수집하는 중...")
            items = driver.find_elements(By.CSS_SELECTOR, "div.item_inner:not(.is-loading)")
            data = []
            
            for idx, item in enumerate(items, 1):
                try:
                    # 동 정보
                    title = item.find_element(By.CSS_SELECTOR, ".item_title .text").text
                    
                    # 가격 정보
                    price_element = item.find_element(By.CSS_SELECTOR, ".price_line")
                    price_type = price_element.find_element(By.CSS_SELECTOR, ".type").text
                    
                    # 기본 가격
                    price = price_element.find_element(By.CSS_SELECTOR, ".price").text
                    
                    # 최고가 존재 여부 확인
                    try:
                        highest_price = price_element.find_element(By.CSS_SELECTOR, ".price.price--highest").text
                        price = f"{price} {highest_price}"
                    except:
                        pass
                    
                    # 상세 정보
                    info = item.find_element(By.CSS_SELECTOR, ".spec").text
                    specs = info.split(", ")
                    
                    # 면적과 층수 추출
                    area = specs[0] if specs else "정보없음"
                    floor = specs[1] if len(specs) > 1 else "정보없음"
                    direction = specs[2] if len(specs) > 2 else "정보없음"
                    
                    # 확인 날짜
                    try:
                        confirm_date = item.find_element(By.CSS_SELECTOR, ".label--confirm .data").text
                    except:
                        confirm_date = "정보없음"
                    
                    # 중개사 수
                    try:
                        agents = item.find_element(By.CSS_SELECTOR, ".label--multicp .count").text
                    except:
                        agents = "1"
                        
                    data.append({
                        "동": title,
                        "거래유형": price_type,
                        "가격": price,
                        "면적": area,
                        "층": floor,
                        "방향": direction,
                        "확인일자": confirm_date,
                        "중개사수": agents
                    })
                    
                    print(f"\r{idx}번째 매물 정보 수집 완료", end="")
                    
                except Exception as e:
                    print(f"\n매물 정보 수집 중 오류 발생: {str(e)}")
                    continue
            
            print("\n\n데이터 정리 중...")
            df = pd.DataFrame(data)
            file_name = f"naver_land_{search_term}_{time.strftime('%Y%m%d_%H%M%S')}.xlsx"
            df.to_excel(file_name, index=False)
            
            print(f"\n수집 완료! 데이터가 {file_name}에 저장되었습니다.")
            print(f"총 {len(data)}개의 매물 정보가 저장되었습니다.\n")
            
            driver.quit()
            
        except Exception as e:
            print(f"오류가 발생했습니다: {str(e)}")
            try:
                driver.quit()
            except:
                pass

def main():
    scraper = NaverLandScraper()
    while True:
        search_term = input("\n검색어를 입력하세요 (종료하려면 'q' 입력): ")
        if search_term.lower() == 'q':
            print("프로그램을 종료합니다.")
            break
        scraper.scrape_data(search_term)

if __name__ == "__main__":
    main()


검색어를 입력하세요 (종료하려면 'q' 입력):  도곡렉슬


브라우저를 시작합니다...
매물 페이지로 이동합니다...
'도곡렉슬' 검색 중...
전체 매물을 불러오는 중...
매물 정보를 수집하는 중...
20번째 매물 정보 수집 완료

데이터 정리 중...

수집 완료! 데이터가 naver_land_도곡렉슬_20241210_070547.xlsx에 저장되었습니다.
총 20개의 매물 정보가 저장되었습니다.



KeyboardInterrupt: Interrupted by user

In [9]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException
import pandas as pd
import time

class NaverLandScraper:
    def __init__(self):
        self.options = webdriver.ChromeOptions()
        self.options.add_argument('--start-maximized')
        
    def click_more_button(self, driver):
        while True:
            try:
                # 페이지 하단으로 스크롤
                driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
                time.sleep(2)
                
                # "더보기" 버튼 찾기 및 클릭
                more_button = driver.find_element(By.CSS_SELECTOR, "button.btn_more")
                driver.execute_script("arguments[0].click();", more_button)
                print("더보기 버튼 클릭...")
                time.sleep(2)
                
            except NoSuchElementException:
                print("더 이상 불러올 매물이 없습니다.")
                break
            except Exception as e:
                print(f"더보기 클릭 중 오류 발생: {str(e)}")
                break

    def scrape_data(self, search_term):
        if not search_term:
            print("검색어를 입력해주세요.")
            return
            
        try:
            print("브라우저를 시작합니다...")
            driver = webdriver.Chrome(options=self.options)
            driver.get("https://land.naver.com/")
            
            # 매물 메뉴 클릭
            print("매물 페이지로 이동합니다...")
            WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.CSS_SELECTOR, "a[href='https://new.land.naver.com/complexes']"))
            ).click()
            
            # 검색창 입력
            print(f"'{search_term}' 검색 중...")
            search_box = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.ID, "land_search"))
            )
            search_box.send_keys(search_term)
            search_box.send_keys(Keys.RETURN)
            
            # 데이터 로딩 대기
            time.sleep(3)
            
            # 모든 매물 로딩
            print("전체 매물을 불러오는 중...")
            self.click_more_button(driver)
            
            # 매물 정보 수집
            print("\n매물 정보를 수집하는 중...")
            items = driver.find_elements(By.CSS_SELECTOR, "div.item_inner:not(.is-loading)")
            data = []
            
            for idx, item in enumerate(items, 1):
                try:
                    # 동 정보
                    title = item.find_element(By.CSS_SELECTOR, ".item_title .text").text
                    
                    # 가격 정보
                    price_element = item.find_element(By.CSS_SELECTOR, ".price_line")
                    price_type = price_element.find_element(By.CSS_SELECTOR, ".type").text
                    
                    # 기본 가격
                    price = price_element.find_element(By.CSS_SELECTOR, ".price").text
                    
                    # 최고가 존재 여부 확인
                    try:
                        highest_price = price_element.find_element(By.CSS_SELECTOR, ".price.price--highest").text
                        price = f"{price} {highest_price}"
                    except:
                        pass
                    
                    # 상세 정보
                    info = item.find_element(By.CSS_SELECTOR, ".spec").text
                    specs = info.split(", ")
                    
                    # 면적과 층수 추출
                    area = specs[0] if specs else "정보없음"
                    floor = specs[1] if len(specs) > 1 else "정보없음"
                    direction = specs[2] if len(specs) > 2 else "정보없음"
                    
                    # 확인 날짜
                    try:
                        confirm_date = item.find_element(By.CSS_SELECTOR, ".label--confirm .data").text
                    except:
                        confirm_date = "정보없음"
                    
                    # 중개사 수
                    try:
                        agents = item.find_element(By.CSS_SELECTOR, ".label--multicp .count").text
                    except:
                        agents = "1"
                        
                    data.append({
                        "동": title,
                        "거래유형": price_type,
                        "가격": price,
                        "면적": area,
                        "층": floor,
                        "방향": direction,
                        "확인일자": confirm_date,
                        "중개사수": agents
                    })
                    
                    print(f"\r{idx}번째 매물 정보 수집 완료", end="")
                    
                except Exception as e:
                    print(f"\n매물 정보 수집 중 오류 발생: {str(e)}")
                    continue
            
            print("\n\n데이터 정리 중...")
            df = pd.DataFrame(data)
            file_name = f"naver_land_{search_term}_{time.strftime('%Y%m%d_%H%M%S')}.xlsx"
            df.to_excel(file_name, index=False)
            
            print(f"\n수집 완료! 데이터가 {file_name}에 저장되었습니다.")
            print(f"총 {len(data)}개의 매물 정보가 저장되었습니다.\n")
            
            driver.quit()
            
        except Exception as e:
            print(f"오류가 발생했습니다: {str(e)}")
            try:
                driver.quit()
            except:
                pass

def main():
    scraper = NaverLandScraper()
    while True:
        search_term = input("\n검색어를 입력하세요 (종료하려면 'q' 입력): ")
        if search_term.lower() == 'q':
            print("프로그램을 종료합니다.")
            break
        scraper.scrape_data(search_term)

if __name__ == "__main__":
    main()


검색어를 입력하세요 (종료하려면 'q' 입력):  도곡렉슬


브라우저를 시작합니다...
매물 페이지로 이동합니다...
'도곡렉슬' 검색 중...
전체 매물을 불러오는 중...
더 이상 불러올 매물이 없습니다.

매물 정보를 수집하는 중...
20번째 매물 정보 수집 완료

데이터 정리 중...

수집 완료! 데이터가 naver_land_도곡렉슬_20241210_070731.xlsx에 저장되었습니다.
총 20개의 매물 정보가 저장되었습니다.



KeyboardInterrupt: Interrupted by user

In [11]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException
import pandas as pd
import time

class NaverLandScraper:
    def __init__(self):
        self.options = webdriver.ChromeOptions()
        self.options.add_argument('--start-maximized')
        
    def scroll_and_extract(self, driver):
        data = []
        last_len = 0
        no_new_items_count = 0
        max_retries = 3  # 새로운 항목이 없을 때 최대 재시도 횟수
        
        while no_new_items_count < max_retries:
            # 현재 보이는 모든 매물 추출
            items = driver.find_elements(By.CSS_SELECTOR, "div.item_inner:not(.is-loading)")
            
            # 새로운 매물이 없으면 카운트 증가
            if len(items) == last_len:
                no_new_items_count += 1
                print(f"새로운 매물 없음... ({no_new_items_count}/{max_retries})")
            else:
                no_new_items_count = 0  # 새로운 매물이 있으면 카운트 리셋
            
            # 새로 추가된 매물만 처리
            for item in items[last_len:]:
                try:
                    # 동 정보
                    title = item.find_element(By.CSS_SELECTOR, ".item_title .text").text
                    
                    # 가격 정보
                    price_element = item.find_element(By.CSS_SELECTOR, ".price_line")
                    price_type = price_element.find_element(By.CSS_SELECTOR, ".type").text
                    price = price_element.find_element(By.CSS_SELECTOR, ".price").text
                    
                    # 최고가 확인
                    try:
                        highest_price = price_element.find_element(By.CSS_SELECTOR, ".price.price--highest").text
                        price = f"{price} {highest_price}"
                    except:
                        pass
                    
                    # 상세 정보
                    info = item.find_element(By.CSS_SELECTOR, ".spec").text
                    specs = info.split(", ")
                    
                    area = specs[0] if specs else "정보없음"
                    floor = specs[1] if len(specs) > 1 else "정보없음"
                    direction = specs[2] if len(specs) > 2 else "정보없음"
                    
                    # 확인 날짜
                    try:
                        confirm_date = item.find_element(By.CSS_SELECTOR, ".label--confirm .data").text
                    except:
                        confirm_date = "정보없음"
                    
                    # 중개사 수
                    try:
                        agents = item.find_element(By.CSS_SELECTOR, ".label--multicp .count").text
                    except:
                        agents = "1"
                        
                    data.append({
                        "동": title,
                        "거래유형": price_type,
                        "가격": price,
                        "면적": area,
                        "층": floor,
                        "방향": direction,
                        "확인일자": confirm_date,
                        "중개사수": agents
                    })
                    
                    print(f"\r현재 {len(data)}개 매물 수집 완료", end="")
                    
                except Exception as e:
                    print(f"\n매물 정보 수집 중 오류: {str(e)}")
                    continue
            
            last_len = len(items)
            
            # 페이지 스크롤
            driver.execute_script("window.scrollBy(0, 500);")
            time.sleep(2)  # 로딩 대기
            
            # 현재 스크롤 위치 확인
            current_scroll = driver.execute_script("return window.pageYOffset;")
            scroll_height = driver.execute_script("return document.documentElement.scrollHeight;")
            
            # 페이지 끝에 도달했는지 확인
            if current_scroll + driver.execute_script("return window.innerHeight;") >= scroll_height:
                if no_new_items_count >= max_retries:
                    print("\n더 이상 새로운 매물이 없습니다.")
                    break
                time.sleep(2)  # 추가 로딩 대기
        
        return data

    def scrape_data(self, search_term):
        if not search_term:
            print("검색어를 입력해주세요.")
            return
            
        try:
            print("브라우저를 시작합니다...")
            driver = webdriver.Chrome(options=self.options)
            driver.get("https://land.naver.com/")
            
            print("매물 페이지로 이동합니다...")
            WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.CSS_SELECTOR, "a[href='https://new.land.naver.com/complexes']"))
            ).click()
            
            print(f"'{search_term}' 검색 중...")
            search_box = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.ID, "land_search"))
            )
            search_box.send_keys(search_term)
            search_box.send_keys(Keys.RETURN)
            
            time.sleep(3)  # 초기 로딩 대기
            
            print("매물 정보 수집을 시작합니다...")
            data = self.scroll_and_extract(driver)
            
            print("\n\n데이터 정리 중...")
            df = pd.DataFrame(data)
            file_name = f"naver_land_{search_term}_{time.strftime('%Y%m%d_%H%M%S')}.xlsx"
            df.to_excel(file_name, index=False)
            
            print(f"\n수집 완료! 데이터가 {file_name}에 저장되었습니다.")
            print(f"총 {len(data)}개의 매물 정보가 저장되었습니다.\n")
            
            driver.quit()
            
        except Exception as e:
            print(f"오류가 발생했습니다: {str(e)}")
            try:
                driver.quit()
            except:
                pass

def main():
    scraper = NaverLandScraper()
    while True:
        search_term = input("\n검색어를 입력하세요 (종료하려면 'q' 입력): ")
        if search_term.lower() == 'q':
            print("프로그램을 종료합니다.")
            break
        scraper.scrape_data(search_term)

if __name__ == "__main__":
    main()


검색어를 입력하세요 (종료하려면 'q' 입력):  도곡렉슬


브라우저를 시작합니다...
매물 페이지로 이동합니다...
'도곡렉슬' 검색 중...
매물 정보 수집을 시작합니다...
새로운 매물 없음... (1/3)
새로운 매물 없음... (2/3)
새로운 매물 없음... (3/3)


데이터 정리 중...

수집 완료! 데이터가 naver_land_도곡렉슬_20241210_070928.xlsx에 저장되었습니다.
총 20개의 매물 정보가 저장되었습니다.



KeyboardInterrupt: Interrupted by user

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import time

class NaverLandScraper:
    def __init__(self):
        self.options = webdriver.ChromeOptions()
        self.options.add_argument('--start-maximized')
        
    def infinite_scroll(self, driver):
        last_height = driver.execute_script("return document.documentElement.scrollHeight")
        while True:
            # 스크롤을 현재 문서 높이만큼 내림
            driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight);")
            
            # 새로운 데이터 로딩 대기
            time.sleep(2)
            
            # 로딩 표시가 사라질 때까지 대기
            loading_elements = driver.find_elements(By.CSS_SELECTOR, ".item_inner.is-loading")
            timeout = 0
            while loading_elements and timeout < 10:
                time.sleep(0.5)
                loading_elements = driver.find_elements(By.CSS_SELECTOR, ".item_inner.is-loading")
                timeout += 1
            
            # 새로운 문서 높이 계산
            new_height = driver.execute_script("return document.documentElement.scrollHeight")
            
            # 더 이상 스크롤이 안되면 종료
            if new_height == last_height:
                break
            last_height = new_height
            print("스크롤 진행 중... 새로운 매물 로딩")

    def extract_items(self, driver):
        items = driver.find_elements(By.CSS_SELECTOR, ".item_inner:not(.is-loading)")
        data = []
        
        for idx, item in enumerate(items, 1):
            try:
                # 기본 정보
                title_elem = item.find_element(By.CSS_SELECTOR, ".item_title .text")
                price_elem = item.find_element(By.CSS_SELECTOR, ".price_line")
                info_elem = item.find_element(By.CSS_SELECTOR, ".info_area")
                
                # 데이터 추출
                title = title_elem.text  # 동
                price_type = price_elem.find_element(By.CSS_SELECTOR, ".type").text  # 거래유형
                price = price_elem.find_element(By.CSS_SELECTOR, ".price").text  # 가격
                
                # 최고가 존재 여부 확인
                try:
                    highest_price = price_elem.find_element(By.CSS_SELECTOR, ".price.price--highest").text
                    price = f"{price} {highest_price}"
                except:
                    pass
                
                # 상세 스펙 정보
                spec = info_elem.find_element(By.CSS_SELECTOR, ".spec").text
                specs = spec.split(", ")
                
                # 면적, 층수, 방향 정보 추출
                area = specs[0] if specs else "정보없음"
                floor = specs[1] if len(specs) > 1 else "정보없음"
                direction = specs[2] if len(specs) > 2 else "정보없음"
                
                # 확인일자
                try:
                    confirm_date = item.find_element(By.CSS_SELECTOR, ".label--confirm .data").text
                except:
                    confirm_date = "정보없음"
                
                # 중개사 수
                try:
                    agents = item.find_element(By.CSS_SELECTOR, ".label--multicp .count").text
                except:
                    try:
                        if item.find_element(By.CSS_SELECTOR, ".agent_name"):
                            agents = "1"
                    except:
                        agents = "정보없음"
                
                data.append({
                    "동": title,
                    "거래유형": price_type,
                    "가격": price,
                    "면적": area,
                    "층": floor,
                    "방향": direction,
                    "확인일자": confirm_date,
                    "중개사수": agents
                })
                
                print(f"\r{idx}번째 매물 정보 수집 완료", end="")
                
            except Exception as e:
                print(f"\n매물 정보 수집 중 오류 발생: {str(e)}")
                continue
                
        return data

    def scrape_data(self, search_term):
        if not search_term:
            print("검색어를 입력해주세요.")
            return
            
        try:
            print("브라우저를 시작합니다...")
            driver = webdriver.Chrome(options=self.options)
            driver.get("https://land.naver.com/")
            
            print("매물 페이지로 이동합니다...")
            WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.CSS_SELECTOR, "a[href='https://new.land.naver.com/complexes']"))
            ).click()
            
            print(f"'{search_term}' 검색 중...")
            search_box = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.ID, "land_search"))
            )
            search_box.send_keys(search_term)
            search_box.send_keys(Keys.RETURN)
            
            # 초기 로딩 대기
            time.sleep(3)
            
            print("\n전체 매물을 불러오는 중...")
            self.infinite_scroll(driver)
            
            print("\n매물 정보 수집을 시작합니다...")
            data = self.extract_items(driver)
            
            print("\n\n데이터 정리 중...")
            df = pd.DataFrame(data)
            file_name = f"naver_land_{search_term}_{time.strftime('%Y%m%d_%H%M%S')}.xlsx"
            df.to_excel(file_name, index=False)
            
            print(f"\n수집 완료! 데이터가 {file_name}에 저장되었습니다.")
            print(f"총 {len(data)}개의 매물 정보가 저장되었습니다.\n")
            
            driver.quit()
            
        except Exception as e:
            print(f"오류가 발생했습니다: {str(e)}")
            try:
                driver.quit()
            except:
                pass

def main():
    scraper = NaverLandScraper()
    while True:
        search_term = input("\n검색어를 입력하세요 (종료하려면 'q' 입력): ")
        if search_term.lower() == 'q':
            print("프로그램을 종료합니다.")
            break
        scraper.scrape_data(search_term)

if __name__ == "__main__":
    main()


검색어를 입력하세요 (종료하려면 'q' 입력):  도곡렉슬


브라우저를 시작합니다...
매물 페이지로 이동합니다...
'도곡렉슬' 검색 중...

전체 매물을 불러오는 중...

매물 정보 수집을 시작합니다...
20번째 매물 정보 수집 완료

데이터 정리 중...

수집 완료! 데이터가 naver_land_도곡렉슬_20241210_071053.xlsx에 저장되었습니다.
총 20개의 매물 정보가 저장되었습니다.

